In [1]:
library(tidyr)
library(dplyr)
library(corrplot)


lifeData_original <- read.csv("D:/Code/BIOSTAT682/Life Expectancy.csv")

countries_to_drop = c("Equatorial Guinea", "Haiti")  # they only have predict data, no training data
lifeData_full <- lifeData_original %>% drop_na() %>% filter(!Country %in% countries_to_drop)
dim(lifeData_original)
dim(lifeData_full)

head(lifeData_full)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


corrplot 0.92 loaded



[1] 2938   22

[1] 1646   22

,Country,Year,Status,Life.expectancy,Adult.Mortality,infant.deaths,Alcohol,percentage.expenditure,Hepatitis.B,Measles,⋯,Polio,Total.expenditure,Diphtheria,HIV.AIDS,GDP,Population,thinness.1.19.years.,thinness.5.9.years.,Income.composition.of.resources,Schooling
,<chr>,<int>,<chr>,<dbl>,<int>,<int>,<dbl>,<dbl>,<int>,<int>,⋯,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Afghanistan,2015,Developing,65.0,263,62,0.01,71.279624,65,1154,⋯,6,8.16,65,0.1,584.25921,33736494,17.2,17.3,0.479,10.1
2,Afghanistan,2014,Developing,59.9,271,64,0.01,73.523582,62,492,⋯,58,8.18,62,0.1,612.69651,327582,17.5,17.5,0.476,10.0
3,Afghanistan,2013,Developing,59.9,268,66,0.01,73.219243,64,430,⋯,62,8.13,64,0.1,631.74498,31731688,17.7,17.7,0.470,9.9
4,Afghanistan,2012,Developing,59.5,272,69,0.01,78.184215,67,2787,⋯,67,8.52,67,0.1,669.95900,3696958,17.9,18.0,0.463,9.8
5,Afghanistan,2011,Developing,59.2,275,71,0.01,7.097109,68,3013,⋯,68,7.87,68,0.1,63.53723,2978599,18.2,18.2,0.454,9.5
6,Afghanistan,2010,Developing,58.8,279,74,0.01,79.679367,66,1989,⋯,66,9.20,66,0.1,553.32894,2883167,18.4,18.4,0.448,9.2


In [2]:
lifeData_train <- lifeData_full %>% filter(Year < 2013)
lifeData_test <- lifeData_full %>% filter(Year >= 2013)
dim(lifeData_train)
dim(lifeData_test)

[1] 1386   22

[1] 260  22

In [3]:
library(R2jags)

status = rep(0,nrow(lifeData_train))

#set up dummy/index variables
#years 2000-2015, 16 year variables, all 1 = 2015
for (i in 1:nrow(lifeData_train)) {
  status[i] = ifelse(lifeData_train$Status[i]=="Developing",1,0)
}

mortality = lifeData_train$"Adult.Mortality"
infant_deaths = lifeData_train$"infant.deaths"
alcohol = lifeData_train$"Alcohol"
percExp = lifeData_train$"percentage.expenditure"
hepB = lifeData_train$"Hepatitis.B"
measles = lifeData_train$"Measles"
BMI = lifeData_train$"BMI"
under5Deaths = lifeData_train$"under.five.deaths"
polio = lifeData_train$"Polio"
totalExp = lifeData_train$"Total.expenditure"
dipth = lifeData_train$"Diphtheria"
hivaids = lifeData_train$"HIV.AIDS"
gdp = lifeData_train$"GDP"
pop = lifeData_train$"Population"
thin1019 = lifeData_train$"thinness.1.19.years"
thin59 = lifeData_train$"thinness.5.9.years"
incomeComp = lifeData_train$"Income.composition.of.resources"
school = lifeData_train$"Schooling"
y = lifeData_train$"Life.expectancy"
N = length(lifeData_train$Country) #go through each point
J = length(unique(lifeData_train$Country)) #go through each country

# encode country
indicesDict = c(unique(lifeData_train$Country))
indices = rep(0,N)
for (i in 1:N) {
  indices[i] = which(indicesDict==lifeData_train$Country[i])
}

Warning message:
"package 'R2jags' was built under R version 4.2.1"
Loading required package: rjags

Warning message:
"package 'rjags' was built under R version 4.2.1"
Loading required package: coda

Warning message:
"package 'coda' was built under R version 4.2.1"
Linked to JAGS 4.3.1

Loaded modules: basemod,bugs


Attaching package: 'R2jags'


The following object is masked from 'package:coda':

    traceplot




## Model 1

In [8]:
life_expectancy.model.1 = function()
{
  for (i in 1:N) {
     y[i] ~ dnorm(mu[indices[i],i],tau) #could do tau*k bc precision
     #indices[i] gets the cluster number
     mu[indices[i],i] <- 
          betay[indices[i],1] #intercept
          + betay[indices[i],2]*mortality[i] #mortality
          + betay[indices[i],3]*infant_deaths[i] #infant_deaths
          + betay[indices[i],4]*alcohol[i] #alcohol
          + betay[indices[i],5]*percExp[i] #percentage expenditure
          + betay[indices[i],6]*hepB[i] #hepatitis B
          + betay[indices[i],7]*measles[i] #meases
          + betay[indices[i],8]*BMI[i] #BMI
          + betay[indices[i],9]*under5Deaths[i] #under 5 deaths
          + betay[indices[i],10]*polio[i] #Polio
          + betay[indices[i],11]*totalExp[i] #total expenditure
          + betay[indices[i],12]*dipth[i] #diptheria
          + betay[indices[i],13]*hivaids[i] #hiv.aids
          + betay[indices[i],14]*gdp[i] #gdp
          + betay[indices[i],15]*pop[i] #population
          + betay[indices[i],16]*thin1019[i] #thinness relevance 10-19 yrs old
          + betay[indices[i],17]*thin59[i] #thinness relevance 5-9 yrs old
          + betay[indices[i],18]*incomeComp[i] #income composition
          + betay[indices[i],19]*school[i] #years of schooling
          + betay[indices[i],20]*status[i] #status, developing or not
  }
  for (j in 1:J) {
    for (k in 1:20) {
      betay[j,k] ~ dnorm(beta[k],tau2)
    }
  }      
  beta[1] ~ dnorm(50, 1e-2)
  for (k in 2:20) {
    beta[k] ~ dnorm(0,1e-6)
  }
  tau ~ dgamma(1e-6,1e-6)
  tau2 ~ dgamma(1e-6,1e-6)
}

In [5]:
save.parms.1 = c("beta","betay")
life_expectancy.data.1 = c("y","mortality","infant_deaths","alcohol", "status",
                            "percExp","hepB","measles","BMI","under5Deaths","polio","totalExp",
                            "dipth","hivaids","gdp","pop","thin1019","thin59",
                            "incomeComp","school","N","J","indices")

life_expectancy.data.1.nocorr = c("y","mortality","alcohol", "status",
                            "percExp","hepB","measles","BMI","under5Deaths","polio","totalExp",
                            "dipth","hivaids","gdp","pop","thin1019","thin59",
                            "incomeComp","school","N","J","indices")                            

In [9]:
life_expectancy.out.1 = jags(data=life_expectancy.data.1,parameters.to.save=save.parms.1,model.file=life_expectancy.model.1,
                           n.chains=3,n.iter=10000,n.burnin=2000,n.thin=1)
print(life_expectancy.out.1, digits = 3)

Compiling model graph
   Resolving undeclared variables
   Allocating nodes
Graph information:
   Observed stochastic nodes: 1386
   Unobserved stochastic nodes: 2642
   Total graph size: 51553

Initializing model



In [10]:
print(life_expectancy.out.1, digits = 3)

Inference for Bugs model at "F:/temp/RtmpQPFgBh/model7b4e8724a3.txt", fit using jags,
 3 chains, each with 10000 iterations (first 2000 discarded)
 n.sims = 24000 iterations saved
               mu.vect sd.vect     2.5%      25%      50%      75%    97.5%
beta[1]         50.216   0.269   49.764   49.874   50.350   50.445   50.501
beta[2]         -0.003   0.005   -0.006   -0.004   -0.003   -0.002    0.001
beta[3]          0.016   0.022   -0.026    0.003    0.020    0.031    0.049
beta[4]          0.418   0.114    0.111    0.386    0.449    0.487    0.562
beta[5]         -0.001   0.005   -0.004   -0.002   -0.001    0.001    0.003
beta[6]          0.000   0.006   -0.009   -0.003    0.000    0.003    0.009
beta[7]          0.000   0.006   -0.003   -0.001    0.000    0.001    0.003
beta[8]         -0.002   0.007   -0.013   -0.006   -0.002    0.002    0.009
beta[9]         -0.062   0.015   -0.087   -0.071   -0.064   -0.052   -0.035
beta[10]        -0.005   0.006   -0.015   -0.008   -0.005   